In [12]:
import os
import pandas as pd
import psycopg2

In [13]:
# PostgreSQL database connection details
db_config = {
    'dbname': 'bicourse_db_41',
    'user': 'bicourse_user_41',
    'password': '860qNdQr83',
    'host': 'mtas01.vlba.uni-oldenburg.de',
    'port': 5432
}

In [ ]:
# Establish database connection
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("✅ Successfully connected to the database.")
except Exception as e:
    print(f"❌ Database connection error: {e}")
    exit()


In [10]:
# Folder containing the CSV files
data_folder = 'uol-bibliography-data-2008-2017-raw-3'

In [ ]:
# Reeating in whole folder for each CSV file
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        print(f"processing: {filename}")
        
        # read  CSV files
        try:
            data = pd.read_csv(file_path)
            # Check columns for discrepancies
            print(f"columns of csv file  {filename}: {data.columns}")
        except Exception as e:
            print(f"error {filename}: {e}")
            continue

        # Remove extra spaces from column names
        data.columns = data.columns.str.strip()

        # Convert the 'Meldetag' column to the correct date format (YYYY-MM-DD)
        try:
            data['Meldetag'] = pd.to_datetime(data['Meldetag'], format='%d.%m.%y').dt.strftime('%Y-%m-%d')
        except Exception as e:
            print(f"error in formating file{filename}: {e}")
            continue

        #  PostgreSQL database table creation
        for _, row in data.iterrows():
            try:
                cursor.execute("""
                    INSERT INTO publications (Fach, "Autor/in", Titel, Typ, Meldetag, Punktzahl, ZahlOldenburgerAutoren, Jahr)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (row['Fach'], row['Autor/in'], row['Titel'], row['Typ'], row['Meldetag'], row['Punktzahl'], row['ZahlOldenburgerAutoren'], row['Jahr']))
            except Exception as e:
                print(f"error in data importing file: {filename}")

        # save changes
        conn.commit()
        print(f" {filename} import.")

#connect to dataset
cursor.close()
conn.close()
print(" Connected and finished. ")